# Create dice objects to be used in simple dice game simulations

In [13]:
import numpy as np

class Die:
    '''
    Dice have a number of sides and, unless in motion, always have one face up.  
    
    Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
    '''
    
    def __init__(self,sides):
        self.sides = sides
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
    def roll(self):
        self.face_up = np.random.choice([i+1 for i in range(self.sides)])
        
        return self.face_up

In [14]:
die = Die(sides=6)

In [3]:
die.face_up

2

In [4]:
die.roll()

5

In [5]:
die.face_up

5

In [141]:
import numpy as np

class Dice_Game:
    '''
    This simple multiplayer dice game is just called "Dice".  
    
    Instructions:
    
    Players determine who goes first, and proceed taking turns in an order (typically clockwise around a table).
    
    Each player, in turn, proceeds by first shaking all six dice, 
    and proceeding along a decision tree of subsequent rolls.
    This tree ends in either a zero score, where a shake results in no dice count for points, 
    or the player "keeps" a number of dice which count for points.
    
    The point value of dice are determined per shake, 
    and point values do not change when mixed with dice kept from previous rolls.
    
    COUNTERS:
    
    One: 100
    Five: 50
    
    THREE-OF-A-KIND:
    Ones: 1000
    Sixes: 600
    Fives: 500
    Fours: 400
    Threes: 300
    Twos: 200
    
    STRAIGHT (1,2,3,4,5,6): 2500
    
    FLUSH (ALL-OF-A-KIND): Instant Win.
    
    On the occasion where all dice rolled are "counters", the dice must all be brought back and shook again.
    The turn proceeds as usual, adding the new shake points to the previous all-counter results.
    
    
    '''
    
    def __init__(self, players, num_dice=6):
        self.player_names = ['Player_{}'.format(name + 1) for name in [j for j in range(players)]]
        self.players = [self.Player(name=player) for player in self.player_names]
        self.dice = [self.Die(sides=6) for die in [i for i in range(num_dice)]]
        self.play_to = 7500
        
        
    def start():
        self.score = score
        
    
    def shake(self,Player):
        shook_dice = [die.roll() for die in self.dice]
        print('I,', Player.name, 'shook the dice:', [i.face_up for i in self.dice])
        #print('shook dice:', shook_dice)
        return shook_dice

        
        
    def player_take_turn(self,Player):
        hand = self.shake(Player)
        counters = []
        
        Player.turn += 1
        
        
    
    class Die:
        '''
        Dice have a number of sides and, unless in motion, always have one face up.  

        Cocked dice are ignored, and motion/trajectories are assumed to be a random choice from the number of sides.
        '''

        def __init__(self,sides):
            self.sides = sides
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

        def roll(self):
            self.face_up = np.random.choice([i+1 for i in range(self.sides)])

            return self.face_up
    
    # Players as subclass, called in Dice_Game init
    class Player:
        '''
        Each player is equipped with a turn method which includes shaking dice, and records their score.
        '''
    
        def __init__(self,name):
            self.name = name
            self.turn = 0
            self.score = 0

            


In [142]:
game = Dice_Game(players=3)

In [143]:
game.play_to

7500

In [144]:
print([i.face_up for i in game.dice])

[1, 3, 1, 1, 4, 4]


In [145]:
game.players[0].name

'Player_1'

In [146]:
game.shake(game.players[0])

I, Player_1 shook the dice: [3, 6, 4, 5, 3, 4]


[3, 6, 4, 5, 3, 4]

In [149]:
game.player_take_turn(game.players[0])

I, Player_1 shook the dice: [1, 5, 2, 2, 4, 4]


In [150]:
game.players[0].turn

1